# EXERCISE 3 - ENERGINET OPTIMAL POWER FLOW PROBLEM

## Imports

In [2]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

## Statement

We consider a 3-bus power system comprising 3 generators: G1 at node n1 , G2 at node n2 ,
G3 at node n3 , and 1 inflexible load: D1 at node n3.

   - c: cost
   - q: max capacity
   - x: reactance
   - Load: inflexible load
  - p: production

Costs 3 generators

In [3]:
G1_c = 70
G2_c = 15
G3_c = 150

Max capacity 3 generators

In [4]:
G1_q = 150
G2_q = 150
G3_q = 150

Inflexible load

In [5]:
Load = 200

Reactance 3 lines

In [6]:
L12_x = 0.4
L23_x = 0.4
L13_x = 0.4

Max flow 3 lines

In [7]:
L12_q = 150
L23_q = 150
L13_q = 150

## Resolution

### a)

Formulate the optimal power flow problem in this system, using the DC linearization of the power flow equations. Specify the number of variables and constraints.

In [9]:
model = gp.Model("Ex3")

#Variables generator productions
G1_p = model.addVar(name='Production Generator 1')
G2_p = model.addVar(name='Production Generator 2')
G3_p = model.addVar(name='Production Generator 3')

#Variables node angles
n1_a = model.addVar(name='Angle node 1') # Reference angle
n2_a = model.addVar(name='Angle node 2')
n3_a = model.addVar(name='Angle node 3')


#Constrains nodes
n1_cons = model.addLConstr(1/(L12_x)*(n1_a - n2_a) + 1/(L13_x)*(n1_a - n3_a), GRB.EQUAL, G1_p)
n2_cons = model.addLConstr(1/(L12_x)*(n1_a - n2_a) - 1/(L23_x)*(n2_a - n3_a), GRB.EQUAL, G2_p)
n3_cons = model.addLConstr(-1/(L13_x)*(n1_a - n3_a) - 1/(L23_x)*(n2_a - n3_a) + Load, GRB.EQUAL, G3_p)

#Constrains lines
L12_consUP = model.addLConstr(1/(L12_x)*(n1_a - n2_a), GRB.LESS_EQUAL, L12_q)
L12_consDOWN = model.addLConstr(1/(L12_x)*(n1_a - n2_a), GRB.GREATER_EQUAL, -L12_q)
L23_consUP = model.addLConstr(1/(L23_x)*(n2_a - n3_a), GRB.LESS_EQUAL, L23_q)
L23_consDOWN = model.addLConstr(1/(L23_x)*(n2_a - n3_a), GRB.GREATER_EQUAL, -L23_q)
L13_consUP = model.addLConstr(1/(L13_x)*(n1_a - n3_a), GRB.LESS_EQUAL, L13_q)
L13_consDOWN = model.addLConstr(1/(L13_x)*(n1_a - n3_a), GRB.GREATER_EQUAL, -L13_q)

#Constrains Generator max power
G1_consUP = model.addLConstr(G1_p, GRB.GREATER_EQUAL, 0)
G1_consDOWN = model.addLConstr(G1_p, GRB.LESS_EQUAL, G1_q)
G2_consUP = model.addLConstr(G2_p, GRB.GREATER_EQUAL, 0)
G2_consDOWN = model.addLConstr(G2_p, GRB.LESS_EQUAL, G2_q)
G3_consUP = model.addLConstr(G3_p, GRB.GREATER_EQUAL, 0)
G3_consDOWN = model.addLConstr(G3_p, GRB.LESS_EQUAL, G3_q)

#Constrains nodes angles
#n1_a_consUP = model.addLConstr(n1_a, GRB.GREATER_EQUAL, - np.pi)
#n1_a_consDOWN = model.addLConstr(n1_a, GRB.LESS_EQUAL, np.pi)
n2_a_consUP = model.addLConstr(n2_a, GRB.GREATER_EQUAL, - np.pi)
n2_a_consDOWN = model.addLConstr(n2_a, GRB.LESS_EQUAL, np.pi)
n3_a_consUP = model.addLConstr(n3_a, GRB.GREATER_EQUAL, - np.pi)
n3_a_consDOWN = model.addLConstr(n3_a, GRB.LESS_EQUAL, np.pi)

#Constrain reference angle, reference angle chosen n1_a
ref_a_cons = model.addLConstr(n1_a, GRB.EQUAL, 0)


#Objective Function
model.setObjective(G1_c*G1_p + G2_c*G2_p + G3_c*G3_p, GRB.MINIMIZE)

model.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-10700T CPU @ 2.00GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 20 rows, 6 columns and 35 nonzeros
Model fingerprint: 0x4a212a48
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [2e+01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 2e+02]
Presolve removed 19 rows and 5 columns
Presolve time: 0.00s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Infeasible model


In [9]:
if model.status == GRB.OPTIMAL:
    optimal_objective = model.ObjVal
    optimal_x_G1 = x_G1.x
    optimal_x_G2 = x_G2.x
    optimal_x_G3 = x_G3.x
    optimal_dual_1 = constraint_1.Pi
    optimal_dual_2 = constraint_2.Pi
    optimal_dual_3 = constraint_3.Pi
    optimal_dual_4 = constraint_4.Pi
    print(f"optimal objective: {optimal_objective}")
    print(f"optimal value of {x_G1.VarName}: {optimal_x_G1}")
    print(f"optimal value of {x_G2.VarName}: {optimal_x_G2}")
    print(f"optimal value of {x_G3.VarName}: {optimal_x_G3}")
else:
    print(f"optimization of {model.ModelName} was not successful")

3.141592653589793

In [4]:
import gurobipy as gp
from gurobipy import GRB

# Parameters
G1_c = 70
G2_c = 15
G3_c = 150

G1_q = 150
G2_q = 150
G3_q = 150

Load = 200

L12_x = 0.4
L23_x = 0.4
L13_x = 0.4

L12_q = 150 
L23_q = 150
L13_q = 150

# Create a new Gurobi model
m = gp.Model("Optimal Power Flow")

# Decision variables
P_G1 = m.addVar(lb=0, ub=G1_q, name="P_G1")  # Power generation by G1
P_G2 = m.addVar(lb=0, ub=G2_q, name="P_G2")  # Power generation by G2
P_G3 = m.addVar(lb=0, ub=G3_q, name="P_G3")  # Power generation by G3
theta_2 = m.addVar(lb=-GRB.INFINITY, name="theta_2")  # Phase angle at n2
theta_3 = m.addVar(lb=-GRB.INFINITY, name="theta_3")  # Phase angle at n3

# Power flows between buses as decision variables
P_12 = m.addVar(lb=-L12_q, ub=L12_q, name="P_12")  # Power flow from n1 to n2
P_23 = m.addVar(lb=-L23_q, ub=L23_q, name="P_23")  # Power flow from n2 to n3
P_13 = m.addVar(lb=-L13_q, ub=L13_q, name="P_13")  # Power flow from n1 to n3

# Objective: Minimize total generation cost
m.setObjective(G1_c * P_G1 + G2_c * P_G2 + G3_c * P_G3, GRB.MINIMIZE)

# Constraints
# Power balance at bus n1 (n1 has G1 and lines n1-n2, n1-n3)
m.addConstr(P_G1 == P_12 + P_13, "Power_balance_n1")

# Power balance at bus n2 (n2 has G2, line n1-n2, and line n2-n3)
m.addConstr(P_G2 == P_12 - P_23, "Power_balance_n2")

# Power balance at bus n3 (n3 has G3, load D1, and lines n1-n3 and n2-n3)
m.addConstr(P_G3 + P_23 + P_13 == Load, "Power_balance_n3")

# Power flow equations (DC power flow approximation)
m.addConstr(P_12 == (0 - theta_2) / L12_x, "Flow_n1_n2")  # Theta_1 is 0 (reference bus)
m.addConstr(P_23 == (theta_2 - theta_3) / L23_x, "Flow_n2_n3")
m.addConstr(P_13 == (0 - theta_3) / L13_x, "Flow_n1_n3")  # Theta_1 is 0

# Generation limits (already implicitly defined by lb/ub in decision variables)
m.addConstr(P_G1 <= G1_q, "Gen_limit_G1")
m.addConstr(P_G2 <= G2_q, "Gen_limit_G2")
m.addConstr(P_G3 <= G3_q, "Gen_limit_G3")

# Line flow limits are automatically handled by the bounds in P_12, P_23, and P_13

# Solve the model
m.optimize()

# Check if the solution is optimal
if m.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    print(f"Generation by G1: {P_G1.X} MW")
    print(f"Generation by G2: {P_G2.X} MW")
    print(f"Generation by G3: {P_G3.X} MW")
    print(f"Power flow from n1 to n2: {P_12.X} MW")
    print(f"Power flow from n2 to n3: {P_23.X} MW")
    print(f"Power flow from n1 to n3: {P_13.X} MW")
    print(f"Total cost: {m.objVal}")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))
CPU model: Intel(R) Core(TM) i7-10700T CPU @ 2.00GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 9 rows, 8 columns and 19 nonzeros
Model fingerprint: 0x1e6c83da
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [2e+01, 2e+02]
  Bounds range     [2e+02, 2e+02]
  RHS range        [2e+02, 2e+02]
Presolve removed 9 rows and 8 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.8000000e+04   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.800000000e+04
Optimal solution found:
Generation by G1: 150.0 MW
Generation by G2: 0.0 MW
Generation by G3: 50.0 MW
Power flow from n1 to n2: 50.0 MW
Power flow from n2 to n3: 50.0 MW
Power flow from n

## b)

Are the solutions of the Economic Dispatch (ED) problem from Exercise 2 feasible for this DC power flow equations? If not, which constraints are violated?